In [57]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Spark_LR_65g").getOrCreate()
spark.sparkContext._conf.getAll()


[('spark.history.kerberos.keytab', 'none'),
 ('spark.ui.proxyBase', '/proxy/application_1548786446322_4794'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.history.ui.port', '18081'),
 ('spark.driver.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.executor.memory', '5g'),
 ('spark.driver.memory', '5g'),
 ('spark.app.name', 'Spark_LR_65g'),
 ('spark.cores.max', '4'),
 ('spark.executor.cores', '4'),
 ('spark.executor.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.port.maxRetries', '128'),
 ('spark.app.id', 'application_1548786446322_4794'),
 ('spark.history.provider',
  'org.apache.spark.deploy.history.FsHistoryProvider'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.yarn.historyServer.address', 'hdp001.cac.queensu.ca:18081'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.filters',
  'org.apache.hadoop

In [59]:

#Update the default configurations
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '65g'), ('spark.app.name', 'LR_unbalanced'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','65g')])
#Stop the current Spark Session
spark.sparkContext.stop()

#Create a Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [5]:

#Update the default configurations
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '5g'), ('spark.app.name', 'LR_imbalanced'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','5g')])
#Stop the current Spark Session
spark.sparkContext.stop()

#Create a Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [61]:
spark.sparkContext._conf.getAll()

[('spark.history.kerberos.keytab', 'none'),
 ('spark.ui.proxyBase', '/proxy/application_1548786446322_4794'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.executor.memory', '65g'),
 ('spark.history.ui.port', '18081'),
 ('spark.driver.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.app.name', 'LR_unbalanced'),
 ('spark.cores.max', '4'),
 ('spark.executor.cores', '4'),
 ('spark.executor.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.port.maxRetries', '128'),
 ('spark.app.id', 'application_1548786446322_4794'),
 ('spark.history.provider',
  'org.apache.spark.deploy.history.FsHistoryProvider'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.yarn.historyServer.address', 'hdp001.cac.queensu.ca:18081'),
 ('spark.driver.memory', '65g'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.filters',
  'org.apache.had

In [62]:
#### Queen's cluster
#https://uoft.cac.queensu.ca/

# LARGE, IMBALANCED dataset
#file_location = "data_reduced_reweighted_final_3_csv"
file_location = "/user/mie_sbetancourt/PROJECT/Data/data_reduced_reweighted_FINAL_3.csv"
#print(file_location)
# SMALL, BALANCED dataset
#file_location = "wasbs:///mie1628-2019-06-18t21-07-05-930z@mie1628.blob.core.windows.net/MIE1628/data_SMALL_balanced.csv/part-00000-0b2544e2-78ed-44d9-808d-1d2e4ec6a8df-c000.csv"

In [63]:
# Import libraries

#data = spark.table("data_reduced_reweighted_final_3_csv")

from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.types import DoubleType, StringType, IntegerType

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SVC_1').getOrCreate()

# Import the data into a Spark DataFrame with the schema 
data = spark.read.format("csv").option("header","true").option("inferSchema","true").load(file_location)


#data = spark.read.csv(file_location, header=True, inferSchema=True)
data = (data.drop("Census_FirmwareVersionIdentifier")
        .withColumn("AVProductsEnabled", when(data["AVProductsEnabled"]=="unknown", -1).otherwise(data["AVProductsEnabled"]))
        .withColumn("classWeightCol", when(data["classWeightCol"]>.5, 0.091).otherwise(0.908)))

data_1 = (data.withColumn("AVProductsEnabled", data["AVProductsEnabled"].cast(IntegerType()))
          .withColumn("AvSigVersion_new", data["AvSigVersion_new"].cast(StringType()))
          .withColumn("Census_OSBuildNumber", data["Census_OSBuildNumber"].cast(StringType()))
          .withColumn("Census_OSBuildRevision", data["Census_OSBuildRevision"].cast(StringType()))
          .withColumn("Census_OSUILocaleIdentifier", data["Census_OSUILocaleIdentifier"].cast(StringType()))
          .withColumn("Census_OSVersion_new", data["Census_OSVersion_new"].cast(StringType()))
          .withColumn("CountryIdentifier", data["CountryIdentifier"].cast(StringType()))
          .withColumn("LocaleEnglishNameIdentifier", data["LocaleEnglishNameIdentifier"].cast(StringType()))
          .withColumn("OsBuild", data["OsBuild"].cast(StringType()))
          .withColumn("OsSuite", data["OsSuite"].cast(StringType())))
data_1 = data_1.withColumnRenamed("HasDetections","label").drop("OsBuildLab_new")
#sampling_seed=1111
#data = data_3.sampleBy("HasDetections", fractions={0: .025, 1: .25}, seed=sampling_seed)
#data_reduced.groupBy("HasDetections").count().show()

In [9]:
#data_reduced.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("wasbs:///mie1628-2019-06-18t21-07-05-930z@mie1628.blob.core.windows.net/MIE1628/data_SMALL_balanced.csv")

In [10]:
#file_location = "wasbs:///mie1628-2019-06-18t21-07-05-930z@mie1628.blob.core.windows.net/MIE1628/data_reduced_reweighted_final_3.csv/part-00000-54f91216-357c-4161-9d25-ee9577740a9c-c000.csv"

#file_location = "wasbs:///mie1628-2019-06-18t21-07-05-930z@mie1628.blob.core.windows.net/MIE1628/data_SMALL_balanced.csv/part-00000-0b2544e2-78ed-44d9-808d-1d2e4ec6a8df-c000.csv"

#data = spark.read.csv(file_location, header=True, inferSchema=True)

In [11]:
#data_1.columns

In [12]:
#data_1.groupBy("label").count().show()

In [13]:
#from pyspark.sql.functions import isnan, when, count, col
### Number of nulls or NAN in each column
#numOfnulls = data_1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data_1.columns])
### View the number of nulls in each column in a Pandas DataFrame format
#numOfnulls_pandas = numOfnulls.toPandas()
#numOfnulls_pandas.T

In [64]:
# This function drops columns which have more than 10% of their entries as null values
import pyspark.sql.functions as F

def drop_null_columns(df):
    """
    This function drops columns which have more than 10% of their entries as null values.
    :param df: A PySpark DataFrame
    """
    null_counts = df.select([F.count(F.when(isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    to_drop = [k for k, v in null_counts.items() if v > 0.1*(df.count())]
    df = df.drop(*to_drop)
    return df

In [15]:
# Drop the colums with more than 10% of the records containing and create a new DataFrame "data_1"
#data_1 = drop_null_columns(data)

In [16]:
# Check the number of columns remaining in DF "data_1"
#len(data_1.columns)

In [17]:
##### Check the number of unique values in each column of the dataframe "data_1" and view in a Pandas DataFrame format
#numUniqueVals = data_1.agg(*(F.countDistinct(col(c)).alias(c) for c in data_1.columns))
#numUniqueVals_pandas = numUniqueVals.toPandas()
#numUniqueVals_pandas.T

In [18]:
#### Create a list of columns which have less than 20 unique values
#numUniqueVals_pandas.T
# Cols_with_less_than_20Unique_vals = [col for col, val in numUniqueVals_pandas.iteritems() if numUniqueVals_pandas[col][0]<20 ]
# Cols_with_less_than_20Unique_vals

In [19]:
# Create a new DF "data_2" which contains the columns with less than 20 unique values only
#data_2 = data_1.select(Cols_with_less_than_20Unique_vals)

In [20]:
# Check the number of columns in DF "data_2"
#len(data_2.columns)

In [21]:
# Sanity check: Check that the number of samples (rows) has not changed from the original dataset 

In [22]:
# Drop the rows (records) with NA entries in any of the X columns of "data_2" and save into a new DF "data_3"
#data_1 = data_1.dropna(how='any')

In [23]:
# Check the number of rows remaining in DF "data_1"
#data_1.select("label").count()

In [24]:
# Create a sample of the data to look at in a Pandas dataframe (the sample contains 5% of the full data)
# data_sample = data_3.sample(False, 0.005)

In [65]:
# Drop the column containing the labels from the features dataframe before vectorizing
data_1b = data_1.drop("label").drop("MachineIdentifier")

In [26]:
data_1.groupBy("label").count().show()

+-----+-------+
|label|  count|
+-----+-------+
|    1| 447000|
|    0|4462591|
+-----+-------+



In [10]:
# List the data types of the columns in DF "data_3b"
#data_1.dtypes

In [66]:
stringCols = []
for col in data_1.dtypes:
    if col[1] == 'string':
        stringCols.append(col[0])

In [67]:
stringCols.pop(0)
#stringCols

'MachineIdentifier'

In [68]:
import numpy as np
numericCols = np.setdiff1d(data_1.columns, stringCols).tolist()
numericCols.remove("MachineIdentifier")
numericCols.remove("classWeightCol")
numericCols.remove("label")

In [ ]:
#numericCols

In [69]:
sampling_seed=1111
trainingData1 = data_1.sampleBy("label", fractions={0: .05, 1: .05}, seed=sampling_seed)
trainingData = trainingData1.sampleBy("label", fractions={0: .9, 1: .9}, seed=sampling_seed)
# Subtracting 'train' from original 'data' to get test set 
testData = trainingData1.subtract(trainingData)

In [33]:
#trainingData.groupBy("label").count().show()

+-----+------+
|label| count|
+-----+------+
|    1| 20030|
|    0|201404|
+-----+------+



In [70]:
import pandas as pd
viewer = trainingData.toPandas()
viewer.head()

,MachineIdentifier,EngineVersion,AppVersion,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,...,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,label,classWeightCol,AvSigVersion_new,OSVer_new,Census_OSVersion_new
0,231532ebd55348282b48b195d3dcd15a,1.1.15200.1,4.18.1807.18075,7,0,53447,1,1,1,66,...,0,0,0,1,5,0,0.091,1.275,10.0.0,10.0
1,0e7a3deed3dc23d01889d1c96f27720f,1.1.15300.6,4.18.1807.18075,7,0,53447,1,1,1,101,...,0,0,0,0,9,0,0.091,1.277,10.0.0,10.0
2,a75b54fa4f54549ebac056b6ce018ba7,1.1.15200.1,4.9.10586.1106,7,0,53447,1,1,1,201,...,0,0,0,0,11,0,0.091,1.275,10.0.0,10.0
3,f94a45006a152df8faa8159aacea3184,1.1.14104.0,4.12.16299.15,7,0,53447,1,1,1,80,...,1,0,1,0,3,0,0.091,1.251,10.0.0,10.0
4,794efec320642a02b18719de96138846,1.1.15100.1,4.9.10586.1106,7,0,53447,1,1,1,29,...,0,0,0,1,10,0,0.091,1.273,10.0.0,10.0


## 1. Logistic Regression

In [71]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('LR_1').getOrCreate()

# Import libraries
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import time
start_time = time.time()

sampling_seed=1111
# The index of string values multiple columns
indexers = [
    StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c),handleInvalid="skip")
    for c in stringCols
]

# The encode of indexed vlaues multiple columns
encoders = [OneHotEncoder(dropLast=False,inputCol=indexer.getOutputCol(),
            outputCol="{0}_encoded".format(indexer.getOutputCol())) 
    for indexer in indexers
]

lr = (LogisticRegression(labelCol="label", featuresCol="features",weightCol="classWeightCol"))

# Vectorizing encoded values
assembler = VectorAssembler(inputCols=([encoder.getOutputCol() for encoder in encoders] + numericCols),outputCol="features")   
      
#pipeline = Pipeline(stages=indexers + encoders+[assembler]+lr)
pipeline = Pipeline(stages=indexers + encoders+[assembler]+[lr])

estimatorParam = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.0, .025, .1, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, .2,.8, 1.0])  \
    .addGrid(lr.fitIntercept,[True,False])  \
    .build()

evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction")
  
crossval = CrossValidator(estimator=pipeline,
                         estimatorParamMaps=estimatorParam,
                         evaluator=evaluator,
                         numFolds=3,
                         parallelism=8,
                         seed=sampling_seed)

cvmodel = crossval.fit(trainingData)      
# Note that the evaluator here is a BinaryClassificationEvaluator and its default metric
# is areaUnderROC.

print("--- %s seconds ---" % (time.time() - start_time))

KeyboardInterrupt: 

In [ ]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
predictions = cvmodel.transform(testData)
train_predictions = cvmodel.transform(trainingData)
#evaluator.evaluate(predictions)

In [ ]:
print("The area under ROC for train set after CV  is {}".format(evaluator.evaluate(train_predictions)))
print("The area under ROC for test set after CV  is {}".format(evaluator.evaluate(predictions)))
print ('Best Param (regParam): ', cvmodel.bestModel.stages[-1]._java_obj.parent().getRegParam())
print ('Best ElasticParam: ', cvmodel.bestModel.stages[-1]._java_obj.parent().getElasticNetParam())

In [ ]:
'''
1st try on Queens
900K training
--- 9758.53647518158 seconds --- 3 hrs
estimatorParam = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.0, .025, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, .5, 1.0])  \

evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction")
  
crossval = CrossValidator(estimator=pipeline,
                         estimatorParamMaps=estimatorParam,
                         evaluator=evaluator,
                         numFolds=3,
                         parallelism=8,
                         seed=sampling_seed)

The area under ROC for train set after CV  is 0.6766547075808887
The area under ROC for test set after CV  is 0.6709478670932243
Best Param (regParam):  0.025
Best ElasticParam:  0.0
                         
2nd try on Queens
200K training                         
--- 4215.349386930466 seconds --- 70 mins
The area under ROC for train set after CV  is 0.6829531111489988
The area under ROC for test set after CV  is 0.6630408598358555
Best Param (regParam):  0.025
Best ElasticParam:  0.0

'''

In [36]:
plotting = predictions.select("label", "prediction", "probability")

In [37]:
#plotting the ROC Curve
trainingSummary = cvmodel.bestModel.stages[-1].summary
roc = trainingSummary.roc.toPandas()
plt.plot([0,1],'r--')
plt.plot(roc['FPR'],roc['TPR'])
plt.ylabel('False Positive Rate')
plt.xlabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

print ('Best Param (regParam): ', cvmodel.bestModel.stages[-1]._java_obj.parent().getRegParam())
print ('Best ElasticParam: ', cvmodel.bestModel.stages[-1]._java_obj.parent().getElasticNetParam())
print("Test set AUROC: {}".format(evaluator.evaluate(predictions)))
print('Training set AUROC: ' + str(trainingSummary.areaUnderROC))

## 1.5 Support Vector Machine

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SVM_1').getOrCreate()

# Import libraries
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import time
start_time = time.time()

sampling_seed=1111
# The index of string values multiple columns
indexers = [
    StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c),handleInvalid="skip")
    for c in stringCols
]

# The encode of indexed vlaues multiple columns
encoders = [OneHotEncoder(dropLast=False,inputCol=indexer.getOutputCol(),
            outputCol="{0}_encoded".format(indexer.getOutputCol())) 
    for indexer in indexers
]

lsvc = (LinearSVC(labelCol="label", featuresCol="features"))
      #,weightCol="classWeightCol")) maxIter=100)) #, regParam=0.1, elasticNetParam=0.5))

# Vectorizing encoded values
assembler = VectorAssembler(inputCols=([encoder.getOutputCol() for encoder in encoders] + numericCols),outputCol="features")   
      
#pipeline = Pipeline(stages=indexers + encoders+[assembler]+lr)
pipeline = Pipeline(stages=indexers + encoders+[assembler]+[lsvc])

estimatorParam = ParamGridBuilder() \
    .addGrid(lsvc.regParam, [0.0, .025, .1]) \
    .addGrid(lsvc.fitIntercept, [True, False])  \
    .build()

evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction")
  
crossval = CrossValidator(estimator=pipeline,
                         estimatorParamMaps=estimatorParam,
                         evaluator=evaluator,
                         numFolds=3,
                         parallelism=8,
                         seed=sampling_seed)

lsvc_cvmodel = crossval.fit(trainingData)      
# Note that the evaluator here is a BinaryClassificationEvaluator and its default metric
# is areaUnderROC.

lsvc_path = temp_path + "/lsvc"
lsvc.save(lsvc_path)
model_path = temp_path + "/lsvc_model"
lsvc_cvmodel.bestModel.save(model_path)


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
###Loading model
from pyspark.ml import *
lsvc_model_path = temp_path + "/lsvc_model"
lsvc_mod2 = PipelineModel.load(lsvc_model_path)
lsvc_predictions = lsvc_mod2.transform(testData)
lsvc_train_predictions = lsvc_mod2.transform(trainingData)
print("The area under ROC for train set after CV  is {}".format(evaluator.evaluate(lsvc_train_predictions)))
print("The area under ROC for test set after CV  is {}".format(evaluator.evaluate(lsvc_predictions)))
print ('Best maxDepth: ', lsvc_mod2.stages[-1]._java_obj.getMaxDepth()
print ('Best maxBins: ', lsvc_mod2.stages[-1]._java_obj.getMaxBins())

## Stratified Split

Would use in imbalanced case but too computationally expensive...

In [39]:
import itertools
import numpy as np

from pyspark import since, keyword_only
from pyspark.ml import Estimator, Model
from pyspark.ml.common import _py2java
from pyspark.ml.param import Params, Param, TypeConverters
from pyspark.ml.param.shared import HasSeed
from pyspark.ml.tuning import CrossValidator, CrossValidatorModel
from pyspark.ml.util import *
from pyspark.ml.wrapper import JavaParams
from pyspark.sql.functions import rand
from functools import reduce

class StratifiedCrossValidator(CrossValidator):
  def stratify_data(self, dataset):
    """
    Returns an array of dataframes with the same ratio of passes and failures.
    Currently only supports binary classification problems.
    """

    epm = self.getOrDefault(self.estimatorParamMaps)
    numModels = len(epm)
    nFolds = self.getOrDefault(self.numFolds)
    split_ratio = 1.0 / nFolds

    passes = dataset[dataset['label'] == 1]
    fails = dataset[dataset['label'] == 0]

    pass_splits = passes.randomSplit([split_ratio for i in range(nFolds)])
    fail_splits = fails.randomSplit([split_ratio for i in range(nFolds)])

    stratified_data = [pass_splits[i].unionAll(fail_splits[i]) for i in range(nFolds)]

    return stratified_data

  def _fit(self, dataset):
    est = self.getOrDefault(self.estimator)
    epm = self.getOrDefault(self.estimatorParamMaps)
    numModels = len(epm)
    eva = self.getOrDefault(self.evaluator)
    nFolds = self.getOrDefault(self.numFolds)
    seed = self.getOrDefault(self.seed)
    metrics = [0.0] * numModels

    stratified_data = self.stratify_data(dataset)

    for i in range(nFolds):
      train_arr = [x for j,x in enumerate(stratified_data) if j != i]
      train = reduce((lambda x, y: x.unionAll(y)), train_arr)
      validation = stratified_data[i]

      models = est.fit(train, epm)

      for j in range(numModels):
        model = models[j]
        metric = eva.evaluate(model.transform(validation, epm[j]))
        metrics[j] += metric/nFolds

    if eva.isLargerBetter():
      bestIndex = np.argmax(metrics)
    else:
      bestIndex = np.argmin(metrics)

    bestModel = est.fit(dataset, epm[bestIndex])
    return self._copyValues(CrossValidatorModel(bestModel, metrics))

## 2. Gradient Boosted Trees

In [ ]:
#from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("lightGBM") \
#            .config("spark.jars.packages", "Azure:mmlspark:0.17") \
#            .getOrCreate()
#
#import mmlspark

In [ ]:
spark = SparkSession.builder.appName("Spark GBT").getOrCreate()

spark.sparkContext.setCheckpointDir('checkpoint/')

temp_path = "/user/mie_sbetancourt/PROJECT/"

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import time
start_time = time.time()

sampling_seed=1111

# The index of string values multiple columns
indexers = [
    StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c),handleInvalid="keep")
    for c in stringCols
]

# The encode of indexed vlaues multiple columns
encoders = [OneHotEncoder(dropLast=False,inputCol=indexer.getOutputCol(),
            outputCol="{0}_encoded".format(indexer.getOutputCol())) 
    for indexer in indexers
]

gbt = (GBTClassifier(labelCol="label", featuresCol="features"))
      
# Vectorizing encoded values
assembler = VectorAssembler(inputCols=([encoder.getOutputCol() for encoder in encoders] + numericCols),outputCol="features")   
      
#pipeline = Pipeline(stages=indexers + encoders+[assembler]+lr)
pipeline = Pipeline(stages=indexers + encoders+[assembler]+[gbt])

estimatorParam = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [4, 8]) \
    .addGrid(gbt.maxBins, [10, 25])  \
    .build()

evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction")
  
gbt_crossval = CrossValidator(estimator=pipeline,
                         estimatorParamMaps=estimatorParam,
                         evaluator=evaluator,
                         numFolds=3,
                         parallelism=8,
                         seed=sampling_seed)

gbt_cvmodel = gbt_crossval.fit(trainingData)      
# Note that the evaluator here is a BinaryClassificationEvaluator and its default metric
# is areaUnderROC.

gbt_path = temp_path + "/gbt"
gbt.save(lr_path)
model_path = temp_path + "/gbt_model"
gbt_cvmodel.bestModel.save(model_path)


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
###Loading model
from pyspark.ml import *
gbt_model_path = temp_path + "/gbt_model"
gbt_mod2 = PipelineModel.load(gbt_model_path)
gbt_predictions = gbt_mod2.transform(testData)
gbt_train_predictions = gbt_mod2.transform(trainingData)
print("The area under ROC for train set after CV  is {}".format(evaluator.evaluate(gbt_train_predictions)))
print("The area under ROC for test set after CV  is {}".format(evaluator.evaluate(gbt_predictions)))
print ('Best maxDepth: ', gbt_mod2.stages[-1]._java_obj.getMaxDepth()
print ('Best maxBins: ', gbt_mod2.stages[-1]._java_obj.getMaxBins())

In [44]:
#plotting the ROC Curve
gbt_trainingSummary = gbt_cvmodel.bestModel.stages[-1].summary
roc = trainingSummary.roc.toPandas()
plt.plot([0,1],'r--')
plt.plot(roc['FPR'],roc['TPR'])
plt.ylabel('False Positive Rate')
plt.xlabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

print ('Best maxBin: ', gbt_cvmodel.bestModel.stages[-1]._java_obj.parent().getMaxBins())
print ('Best maxDepth: ', gbt_cvmodel.bestModel.stages[-1]._java_obj.parent().getMaxDepth())
print("Test set AUROC: {}".format(evaluator.evaluate(gbt_predictions)))
print('Training set AUROC: ' + str(gbt_trainingSummary.areaUnderROC))


NameError: name 'temp_path' is not defined

## 3. Random Forest

In [ ]:
spark = SparkSession.builder.appName("Spark RandFor").getOrCreate()

spark.sparkContext.setCheckpointDir('checkpoint/')

temp_path = "/user/mie_sbetancourt/PROJECT/"

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
#from FeatureImportanceSelector import ExtractFeatureImp, FeatureImpSelector

import time
start_time = time.time()

sampling_seed=1111

# The index of string values multiple columns
indexers = [
    StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c),handleInvalid="keep")
    for c in stringCols
]

# The encode of indexed vlaues multiple columns
encoders = [OneHotEncoder(dropLast=False,inputCol=indexer.getOutputCol(),
            outputCol="{0}_encoded".format(indexer.getOutputCol())) 
    for indexer in indexers
]

randfor = (RandomForestClassifier(labelCol="label", featuresCol="features"))
      
# Vectorizing encoded values
assembler = VectorAssembler(inputCols=([encoder.getOutputCol() for encoder in encoders] + numericCols),outputCol="features")   

#labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
#                                   labels=labelIndexer.labels)

#pipeline = Pipeline(stages=indexers + encoders+[assembler]+lr)
pipeline = Pipeline(stages=indexers + encoders+[assembler]+[randfor])

estimatorParam = ParamGridBuilder() \
    .addGrid(randfor.numTrees, [150,200,250,300]) \
    .addGrid(randfor.maxDepth, [10,20,30]) \
    .addGrid(randfor.maxBins, [20]) \
    .build()

evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction")
  
crossval = CrossValidator(estimator=pipeline,
                         estimatorParamMaps=estimatorParam,
                         evaluator=evaluator,
                         numFolds=3,
                         parallelism=8,
                         seed=sampling_seed)

rand_cvmodel = crossval.fit(trainingData)      
# Note that the evaluator here is a BinaryClassificationEvaluator and its default metric
# is areaUnderROC.

rand_path = temp_path + "/rand"
randfor.save(rand_path)
model_path = temp_path + "/rand_model"
rand_cvmodel.bestModel.save(model_path)


print("--- %s seconds ---" % (time.time() - start_time))

In [32]:
rand_path = temp_path + "/rand"
randfor.save(rand_path)
model_path = temp_path + "/rand_model"
rand_cvmodel.bestModel.save(model_path)

In [33]:
#tried more trees but couldn't handle it
#above took 44 mins
rand_predictions = rand_cvmodel.transform(testData)
rand_train_predictions = rand_cvmodel.transform(trainingData)

In [34]:
#rand_feat_imp = rand_cvmodel.bestModel.stages[-1].featureImportances
print ('Best numTrees: ', rand_cvmodel.bestModel.stages[-1]._java_obj.parent().getNumTrees())
print ('Best maxBins: ', rand_cvmodel.bestModel.stages[-1]._java_obj.parent().getMaxBins())
print ('Best maxDepth: ', rand_cvmodel.bestModel.stages[-1]._java_obj.parent().getMaxDepth())
print("Test set AUROC: {}".format(evaluator.evaluate(rand_train_predictions)))
print('Training set AUROC: {}'.format(evaluator.evaluate(rand_predictions)))


Best numTrees:  150
Best maxBins:  20
Best maxDepth:  10
Test set AUROC: 0.6551282170875691
Training set AUROC: 0.6466502513935741


In [ ]:
from pyspark.ml import *
rand_model_path = temp_path + "/rand_model"
rand_mod2 = PipelineModel.load(rand_model_path)
rand_predictions = rand_mod2.transform(testData)
rand_train_predictions = rand_mod2.transform(trainingData)
print("The area under ROC for train set is {}".format(evaluator.evaluate(rand_train_predictions)))
print("The area under ROC for test set is {}".format(evaluator.evaluate(rand_predictions)))
print ('Best numTrees: ', rand_mod2.stages[-1]._java_obj.getMaxDepth()
print ('Best maxBins: ', rand_mod2.stages[-1]._java_obj.getMaxBins())
print ('Best maxDepth: ', rand_mod2.stages[-1]._java_obj.getMaxDepth())

In [50]:
'''
3-cross val

1st try (Azure)
2296.3062093257904 secs
maxDepth:  8, 10
numTrees: 8, 10
Best maxDepth:  10
Best numTrees: 10
Test set AUROC: 0.6244246230461733
Training set AUROC: 0.6263909435756815


2nd try (Queens)
--- 6261.523066282272 seconds ---
randfor.numTrees, [10, 15]) \
(randfor.maxDepth, [3, 4, 5])
Best numTrees:  15
Best maxDepth:  5
Training set AUROC: 0.6210402526931081
Test set AUROC: 0.6256766022080265


3rd try (Queens)
.addGrid(randfor.numTrees, [50,100]) \
Best numTrees:  100
Best maxBin:  20
Best maxDepth:  7
Test set AUROC: 0.6433332978783131
Training set AUROC: 0.6407116809380157


4th try (Queens)
addGrid(randfor.numTrees, [150,200]) \
Best numTrees:  150
Best maxBins:  20
Best maxDepth:  10
Test set AUROC: 0.6551282170875691


'''

In [27]:
#rand_feat_imp = rand_cvmodel.bestModel.stages[-1].featureImportances

(9050,[0,1,2,3,4,5,6,7,11,14,15,16,61,62,66,67,70,72,73,77,79,86,87,94,161,163,168,169,170,171,174,175,179,181,182,183,187,188,189,191,192,194,198,201,205,226,232,280,296,297,309,319,341,360,370,409,576,644,656,715,870,916,1143,1568,2055,4925,7410,7413,7417,7419,7421,7422,7423,7424,7427,7438,7443,7445,7451,7455,7461,7468,7474,7478,7482,7483,7487,7490,7491,7518,7524,7631,7633,7634,7635,7637,7642,7643,7645,7653,7664,7669,7672,7682,7684,7694,7697,7699,7703,7773,7883,7888,7889,7892,7893,7896,7897,7898,7899,7911,7946,7947,7961,7963,7964,7969,7970,7977,7978,7979,7983,7988,7990,7992,8001,8006,8008,8012,8014,8020,8022,8024,8032,8039,8041,8042,8046,8047,8049,8061,8062,8132,8133,8136,8138,8139,8140,8141,8142,8145,8148,8152,8156,8157,8158,8159,8162,8168,8172,8188,8194,8252,8257,8263,8302,8307,8370,8372,8390,8391,8392,8411,8412,8413,8414,8417,8425,8429,8433,8437,8440,8447,8455,8462,8463,8468,8469,8470,8484,8485,8489,8501,8558,8559,8560,8567,8569,8570,8571,8572,8578,8584,8585,8586,8587,8588,8591,85

In [ ]:
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

In [ ]:
xtractFeatureImp(rand_cvmodel.bestModel.stages[-1].featureImportances, rand_train_predictions, "features").head(10)

In [32]:
''''''
from itertools import chain
attrs = sorted(
    (attr["idx"], attr["name"]) for attr in (chain(*rand_train_predictions
        .schema["features"]
        .metadata["ml_attr"]["attrs"].values())))

rand_feat_imp = [(name, rand_cvmodel.bestModel.stages[-1].featureImportances[idx])
 for idx, name in attrs
 if rand_cvmodel.bestModel.stages[-1].featureImportances[idx]]

In [35]:
[(name, rand_cvmodel.bestModel.stages[-1].pValues[idx]) for idx, name in attrs]

AttributeError: 'RandomForestClassificationModel' object has no attribute 'pValues'

## 4. lightGBM

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Spark lightGBM") \
            .config("spark.jars.packages", "Azure:mmlspark:0.17") \
            .getOrCreate()

spark.sparkContext.setCheckpointDir('checkpoint/')

from pyspark.ml import Pipeline
import mmlspark
from mmlspark import LightGBMClassifier
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from FeatureImportanceSelector import ExtractFeatureImp, FeatureImpSelector

import time
start_time = time.time()

sampling_seed=1111

# The index of string values multiple columns
indexers = [
    StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c),handleInvalid="keep")
    for c in stringCols
]

# The encode of indexed vlaues multiple columns
encoders = [OneHotEncoder(dropLast=False,inputCol=indexer.getOutputCol(),
            outputCol="{0}_encoded".format(indexer.getOutputCol())) 
    for indexer in indexers
]

lgbm = (LightGBMClassifier(labelCol="label", featuresCol="features"))
      
# Vectorizing encoded values
assembler = VectorAssembler(inputCols=([encoder.getOutputCol() for encoder in encoders] + numericCols),outputCol="features")   

#labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
#                                   labels=labelIndexer.labels)

pipeline = Pipeline(stages=indexers + encoders+[assembler]+[lgbm])

estimatorParam = ParamGridBuilder() \
    .addGrid(lgbm.numLeaves, [100,150]) \
    .addGrid(lgbm.maxDepth, [5,7]) \
    #.addGrid(lgbm.bagging_fraction, [.8]) \
    #.addGrid(lgbm.max_bin, [50]) \
    #.addGrid(lgbm.bagging_freq, [10]) \
    .addGrid(lgbm.earlyStoppingRound, [5]) \
    .build()

evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction")
  
crossval = CrossValidator(estimator=pipeline,
                         estimatorParamMaps=estimatorParam,
                         evaluator=evaluator,
                         numFolds=3,
                         parallelism=8,
                         seed=sampling_seed)

lgbm_cvmodel = crossval.fit(trainingData)      
# Note that the evaluator here is a BinaryClassificationEvaluator and its default metric
# is areaUnderROC.

lgbm_path = temp_path + "/lgbm"
lgbm.save(lgbm_path)
model_path = temp_path + "/lgbm_model"
lgbm_cvmodel.bestModel.save(model_path)

print("--- %s seconds ---" % (time.time() - start_time))

ModuleNotFoundError: No module named 'mmlspark'

In [ ]:
'''2nd place lightGBM
best_hyp = {'boosting_type': 'gbdt',
            'class_weight': None,
            'colsample_bytree': 0.2685319471585845,
            'learning_rate': 0.011114918157721438,
            'min_child_samples': 270,
            'num_leaves': 261,
            'reg_alpha': 0.4182767807212193,
            'reg_lambda': 0.07336659149142766,
            'subsample_for_bin': 40000,
            'subsample': 0.6462594904717385}
'''

In [ ]:
###Loading model
from pyspark.ml import *
lgbm_model_path = temp_path + "/lgbm_model"
lgbm_mod2 = PipelineModel.load(lgbm_model_path)
lgbm_predictions = lgbm_mod2.transform(testData)
lgbm_train_predictions = lgbm_mod2.transform(trainingData)
print("The area under ROC for train set is {}".format(evaluator.evaluate(lgbm_train_predictions)))
print("The area under ROC for test set is {}".format(evaluator.evaluate(lgbm_predictions)))
print ('Best maxDepth: ', lgbm_mod2.stages[-1]._java_obj.getMaxDepth()
print ('Best maxBins: ', lgbm_mod2.stages[-1]._java_obj.getMaxBins())